## Questão de Negócio

A empresa Star Jeans! Eduardo e Marcelo são dois brasileiros, amigos e sócios de empreendi-
mento. Depois de vários negócio bem sucedidos, eles estão planejando entrar no mercado de moda
dos USA como um modelo de negócio do tipo E-commerce.
A idéia inicial é entrar no mercado com apenas um produto e para um público específico, no caso
o produto seria calças Jenas para o público masculino. O objetivo é manter o custo de operação
baixo e escalar a medida que forem conseguindo clientes.
Porém, mesmo com o produto de entrada e a audiência definidos, os dois sócios não tem experiência
nesse mercado de moda e portanto não sabem definir coisas básicas como preço, o tipo de calça e
o material para a fabricação de cada peça.
Assim, os dois sócios contrataram uma consultoria de Ciência de Dados para responder as seguintes
perguntas: 

1. Qual o melhor preço de venda para as calças? 

2. Quantos tipos de calças e suas
cores para o produto inicial? 

3. Quais as matérias-prima necessárias para confeccionar as calças?

Obs.: As principais concorrentes da empresa Start Jeans são as americadas H&M e Macys.

In [23]:
data.isna().sum()

product_id            0
product_category      0
product_name          0
product_price         0
scrapy_datetime       0
style_id              0
color_id              0
color_name            1
fit                 261
size_number         261
size_model          261
cotton                0
spandex               0
polyester             0
elastomultiester      0
dtype: int64

# Visualização da Tabela Final

# Import Librarys

In [1]:
import re #regex Library
import warnings
import inflection

import numpy             as np
import pandas            as pd
import seaborn           as sns
import matplotlib.pyplot as plt

from IPython.display       import Image
from IPython.core.display  import HTML


warnings.filterwarnings( 'ignore' )

## Help Function

In [2]:
def jupyter_settings():
    #%matplotlib notebook
    #%pylab inline
    
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [25, 12]
    plt.rcParams['font.size'] = 24
    
    display( HTML( '<style>.container { width:100% !important; }</style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )
    
    sns.set()
    
jupyter_settings()

## Load Data

In [3]:
df = pd.read_csv('../data/data_raw_clean.csv')

In [4]:
df.sample(100)

,Unnamed: 0,product_id,product_category,product_name,product_price,scrapy_datetime,style_id,color_id,color_name,fit,size_number,size_model,cotton,spandex,polyester,elastomultiester
125,125,1024256008,men_jeans_slim,slim_jeans,19.99,2022-04-19 13:17:23,1024256,8,denim_blue,slim_fit,188.0,size 31/32,0.99,0.01,0.00,0.00
101,101,690449051,men_jeans_ripped,skinny_jeans,39.99,2022-04-19 13:17:23,690449,51,black,skinny_fit,187.0,size 32/32,0.98,0.02,0.00,0.00
335,335,985159007,men_jeans_skinny,skinny_jeans,19.99,2022-04-19 13:17:23,985159,7,dark_blue,skinny_fit,187.0,size 31/32,0.99,0.01,0.00,0.00
408,408,811993034,men_jeans_regular,regular_jeans,29.99,2022-04-19 13:17:23,811993,34,dark_blue,NaN,NaN,NaN,0.00,0.00,0.00,0.00
473,473,875105026,men_jeans_relaxed,relaxed_jeans,39.99,2022-04-19 13:17:23,875105,26,dark_denim_blue,NaN,NaN,NaN,0.00,0.00,0.00,0.00
84,84,690449036,men_jeans_ripped,skinny_jeans,39.99,2022-04-19 13:17:23,690449,36,dark_blue_washed_out,skinny_fit,187.0,size 32/32,0.98,0.02,0.00,0.00
475,475,875105026,men_jeans_relaxed,relaxed_jeans,39.99,2022-04-19 13:17:23,875105,26,black,NaN,NaN,NaN,0.00,0.00,0.00,0.00
41,41,938875013,men_jeans_slim,slim_tapered_jeans,39.99,2022-04-19 13:17:23,938875,13,black,slim_fit,187.0,size 31/32,0.98,0.02,0.00,0.00
108,108,979945002,men_jeans_loose,loose_jeans,39.99,2022-04-19 13:17:23,979945,2,black,loose_fit,180.0,size 31/32,1.00,0.00,0.00,0.00
180,180,979945003,men_jeans_loose,loose_jeans,39.99,2022-04-19 13:17:23,979945,3,light_denim_blue,NaN,NaN,NaN,0.00,0.00,0.00,0.00


# Rename columns

In [5]:
data = df.copy()

In [6]:
data.columns

Index(['Unnamed: 0', 'product_id', 'product_category', 'product_name',
       'product_price', 'scrapy_datetime', 'style_id', 'color_id',
       'color_name', 'fit', 'size_number', 'size_model', 'cotton', 'spandex',
       'polyester', 'elastomultiester'],
      dtype='object')

# Clean to dataframe

In [7]:
#data.shape
data.dtypes

Unnamed: 0            int64
product_id            int64
product_category     object
product_name         object
product_price       float64
scrapy_datetime      object
style_id              int64
color_id              int64
color_name           object
fit                  object
size_number         float64
size_model           object
cotton              float64
spandex             float64
polyester           float64
elastomultiester    float64
dtype: object

In [8]:
data.isna().sum()

Unnamed: 0            0
product_id            0
product_category      0
product_name          0
product_price         0
scrapy_datetime       0
style_id              0
color_id              0
color_name            1
fit                 261
size_number         261
size_model          261
cotton                0
spandex               0
polyester             0
elastomultiester      0
dtype: int64

In [10]:
# Eliminando linhas em branco
data = data.dropna( subset=['fit'] ).reset_index()

In [11]:
data = data.drop( columns=['Unnamed: 0', 'index'], axis=1 )

In [9]:
#df.isna().sum()
data.shape

(570, 16)

In [12]:
data.shape

(309, 15)

In [13]:
data.head()

,product_id,product_category,product_name,product_price,scrapy_datetime,style_id,color_id,color_name,fit,size_number,size_model,cotton,spandex,polyester,elastomultiester
0,1024256001,men_jeans_slim,slim_jeans,19.99,2022-04-19 13:17:23,1024256,1,denim_blue,slim_fit,188.0,size 31/32,0.99,0.01,0.0,0.0
1,1024256002,men_jeans_slim,slim_jeans,19.99,2022-04-19 13:17:23,1024256,2,denim_blue,slim_fit,188.0,size 31/32,0.99,0.01,0.0,0.0
2,938875002,men_jeans_slim,slim_tapered_jeans,39.99,2022-04-19 13:17:23,938875,2,light_denim_blue,slim_fit,187.0,size 31/32,0.98,0.02,0.0,0.0
3,938875002,men_jeans_slim,slim_tapered_jeans,39.99,2022-04-19 13:17:23,938875,2,white,slim_fit,187.0,size 31/32,0.98,0.02,0.0,0.0
4,938875002,men_jeans_slim,slim_tapered_jeans,39.99,2022-04-19 13:17:23,938875,2,dark_gray,slim_fit,187.0,size 31/32,0.98,0.02,0.0,0.0
